# Multinomial logit model (MNL) for mode choice

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Overview

This notebook covers the estimation of a multinomial logit model in python with maximum likelihhod estimation:

* Load necessary packages
* Define variables and parameters to estimate
* Model specification
* Model output

Have a go at working through the notebook. To run a code cell, just click on it (to see a green box around it) and then press the **Run** button at the top! 

Some cells have blank lines for you to complete. There is always a comment telling you what to do!

You can also add a new cell by pressing the **+** button at the top of the page.

## Context: Crowdshipping

We have a database of crowdshippers acceptance or rejection of suggested routes. Each crowdshipper is evaluating based on variables such as parcel size, fragility, time of the day, travel time, compensation etc.

**Task: Specify and estimate a binary choice model using all variables in the data set as explanatory variables**

## Model specification

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [1]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
from scipy.optimize import minimize # optimisation routine for parameter estimation
from scipy.stats import norm # normal distribution density function
import numdifftools as nd # we use this to calculate t-ratios and p-values
from scipy.stats import t # we use this to calculate the CFDs needed for t-ratios and p-values
import csv # we need this to store our parameters as csv

### Let's give a name to our model

In [2]:
model_name = 'MNL_crowd_shipping' # Name we want to give to our model (this is used when saving the output)

### Panel structure
We need to define whether our data is panel (i.e. multiple observations per individual) or not

In [3]:
panel = 0 # switch to 1 if data is panel (any other value if not panel)

### Define if we use mixing

In [4]:
mixing = 0 # switch to 1 if we apply mixing (any other value if no mixing applied)

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

**Important!** Make sure the data is in the same folder with the notebook file

In [5]:
# Command to load the data
data = pd.read_table('data_crowdshipping.txt')

# Number of observations (we need this to caclulate goodness-of-fit indices)
Nobs = data.shape[0]

## Print the data

Let's quickly print the data. Simply type *data* in the field below

(If we open our data set with a different name e.g. *database* then the latter should be used)

* Please make sure to change the name througout the whole script if you change the data frame name.

In [6]:
# Type "data" in this field (without the quotation) and run the cell (Shift + return)
data

,ID,trip_duration,time_of_day,Size,Fragile,Compensation,choice
0,1,30.951510,2,0.161859,0,7.240899,1
1,2,23.530198,3,0.250437,0,8.805259,1
2,3,38.612719,1,0.331218,0,10.101263,0
3,4,18.714333,2,0.426646,0,11.268851,0
4,5,34.559455,3,0.055788,1,12.380939,0
...,...,...,...,...,...,...,...
9995,9996,21.443871,2,0.249866,0,16.769894,1
9996,9997,36.608476,3,0.330630,0,18.550652,1
9997,9998,29.285178,4,0.425832,0,21.583600,0
9998,9999,48.568536,1,0.689021,0,6.641191,1


## Print the variable names (columns)

We can also print the variable names only using the piece of code below

* This is useful during model specification to easily access the column names

In [7]:
print(data.columns)

Index(['ID', 'trip_duration', 'time_of_day', 'Size', 'Fragile', 'Compensation',
       'choice'],
      dtype='object')


## Variable definition

We need to define the variables (as numpy arrays) that we will use in our model.

* The arrays can have any name but it is more convenient to use the same name as in the data set.
* The *".reshape(-1, 1)"* is not necessary now but it is required if we include mixing (random effects) in our model.
    * Reshape allows to multiply arrays of different dimensions.

##### Define explanatory (independent) variables

In [8]:
# Example: variable_name = np.array(data['variable_name']).reshape(-1, 1)

choice = np.array(data['choice']).reshape(-1, 1)
trip_duration = np.array(data['trip_duration']).reshape(-1, 1)
time_of_day = np.array(data['time_of_day']).reshape(-1, 1)
size = np.array(data['Size']).reshape(-1, 1)
fragile = np.array(data['Fragile']).reshape(-1, 1)
compensation = np.array(data['Compensation']).reshape(-1, 1)


## Add new variables

What if I want to add headway time for all alternatives?

##### Define the ID variable

In [9]:
# Example: ID = np.array(data['ID']) # ID does not need to be reshaped

ID = np.array(data['ID']) # ID does not need to be reshaped

##### Delete the data frame
*We may want to delete the data frame to preserve some RAM (suggested for very large data sets)*

In [10]:
# Activate to delete the data frame.
# del[data]

## Model specification

We now need to create a function that includes our model.

* Python functions are defined as: def *function_name*(parameters):
* We end a function as: return *value_to_return*

In the current implementation we specify two different functions as:
* *function 1* calculates the log likelihood per observations
* *function 2* calculates the sum of log likelihood taking as input the result from *function 1*

*We define two different functions to be more flexible in the post estimation processing later in the code*

We use some python (numpy) functions such '*exp*' or '*log*'. To execute these in the current example, we need to also call numpy; hence, we have *np.exp()* and *np.log()*.

### Define parameters and starting values

Ultimately, we want to estimate the value of some parameters that maximise the likelihood of our observations of the dependent variable.

Before starting the estimation process, we need to define some starting values for the parameters to be estimated. The typecal starting value is zero.

In [35]:
betas_start = {"asc": 0, "beta_dur": 0, "beta_time": 0, "beta_frag": 0, "beta_size": 0,
              "beta_compen": 0}

### Load old parameter estimates results

Sometimes, we want to use results from old models as starting values.
* To do that, we will load the iteration file from a previous estimation
* Please note that only values of parameters with same names with our current model will be copied

In [36]:
# ### Activate this cell to load old results ###

# # Open old iteration file
# betas_old = pd.read_csv('model_name_iterations.csv') # file with old model iterations (edit accordingly)

# # Keep last row
# betas_old = betas_old.tail(1)

# # Convert to dictionary
# betas_old = betas_old.to_dict(orient='records')[0]

# # Copy values from old to start for keys that are common to both dictionaries
# betas_start = {k: betas_old[k] for k in betas_start.keys() & betas_old.keys()}

# # Delete old estimates
# del[betas_old]

### Function 1: log likelihood (LL)
This function calculates the log likelihood per observation

In [37]:
def LL(betas): # betas is a vector with the parameters we want to estimate
   
    # First let's define the parameters to be estimated.
    # The parameter names are imported directly from 'beta_start' that we defined earlier
    
    for pn in range(len(betas_start.values())):
        globals()[np.array(list(betas_start.keys()))[pn]] = betas[pn]
        
    # Then we need to define the main model specification
    
    # We need to start by defining the utility functions
    # Please make sure that you are using the same names for the parameters as those defined in 'betas_start'
    
    U_yes = asc + beta_dur * trip_duration + beta_time * time_of_day + beta_size * size + beta_compen * compensation + beta_frag * fragile
    U_no = 0

    
    #######################################################################################
    #######################################################################################
    
    # TASK ALERT #1!! How about adding the headway in the utility function?? #

    # TASK ALERT #2!! How about adding alternative specific parameters per travel mode?? #

    #######################################################################################
    #######################################################################################
    
    # And we need to take the exponents of utilities as
    U_yes = np.exp(U_yes)
    U_no = np.exp(U_no)


    
    # We need the sum of all utilities to calculate our choice probabilities
    U_sum = U_yes + U_no
    
    # And then we need to calculate our chouce probabilities
    P_yes = U_yes / U_sum
    P_no = U_no / U_sum

    
    # The total probability is then:
    P = P_yes * (choice==1) + P_no * (choice==0) 
    
    ############################################################################################################
    ############################################################################################################
    # - Now this below is relevant if we have panel data and apply mixing (Do not change this piece of code!) -#
    if panel == 1:
    # Do it as panel
        P = pd.DataFrame(P)
        P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
        P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
        # We take the product of probabilities per individual per draw and then delete the ID column
        P = P.groupby('ID', as_index=False).prod()
        P = P.drop('ID', axis=1)
   
    if mixing == 1:
        # We take the average per row to get the average probability per individual (if mixing == 1)
        
        if panel == 1:
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
        
        if panel == 0:
            P = pd.DataFrame(P)
            P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
            P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
            # We take the product of probabilities per individual per draw and then delete the ID column
            P = P.groupby('ID', as_index=False).prod()
            P = P.drop('ID', axis=1)
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
            
    P = np.array(P)
    ### --- This is where the panel data approach ends. --- ###
    ############################################################################################################
    ############################################################################################################
    
    # We then take the log of the density function
    logprob = np.log(P)
    
    return logprob

### Function 2: sum of log likelihood (SLL)
This function simply takes the sum of log likelihood that we calculated with the first function

In [38]:
def SLL(betas):
    return -sum(LL(betas))

## Model estimation

### Warnings

Sometimes, optimisation procedures may 'overdo' it with warnings during estimation.
We can supress these with the piece of code below (not always advisable)

In [39]:
import warnings
warnings.filterwarnings("ignore")

### Estimation

Now it is finally time to run our estimation command.
We use an optimisation algorith called 'BFGS'.

**Do not to edit the lines of code below (except optimisation routine if needed).**

In [40]:
# This will give us the initial loglikelihood value as an output
def callback1(betas):
    print("Current log likelihood:", -SLL(betas))

# This function will allow as to store parameter estimates during iterations
# Initialise list to store parameter values
parameter_values = [np.array(list(betas_start.values()))]
# Then define the function
def callback2(betas):    
    parameter_values.append(betas)
    column_names = list(betas_start.keys())
    with open(f'{model_name}_iterations.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        writer.writerows(parameter_values)

# Now let's combine the two callback functions
def combined_callback(betas):
    callback1(betas)
    callback2(betas)
        
print("Initial log likelihood:", -SLL(np.array(list(betas_start.values()))))

# Choose optimisation routine (preferably BFGS)
optimiser = 'L-BFGS-B' # BFGS or L-BFGS-B or nelder-mead

result = minimize(SLL, np.array(list(betas_start.values())), method=optimiser,callback=combined_callback, 
                  options={'disp':False}) # ,bounds=bounds1
#args = (parameter_values,)
print("Final log likelihood:", -result.fun)

Initial log likelihood: [-6931.4718056]
Current log likelihood: [-6712.88362878]
Current log likelihood: [-6711.00428678]
Current log likelihood: [-6693.84522733]
Current log likelihood: [-6681.37247202]
Current log likelihood: [-6670.15172559]
Current log likelihood: [-6667.87018597]
Current log likelihood: [-6666.04937523]
Current log likelihood: [-6659.6003103]
Current log likelihood: [-6646.40582791]
Current log likelihood: [-6617.85153512]
Current log likelihood: [-6576.94739755]
Current log likelihood: [-6567.96827651]
Current log likelihood: [-6532.04674645]
Current log likelihood: [-6519.65287566]
Current log likelihood: [-6516.5626133]
Current log likelihood: [-6514.22918187]
Current log likelihood: [-6507.53810854]
Current log likelihood: [-6492.6558638]
Current log likelihood: [-6462.23720958]
Current log likelihood: [-6425.5847937]
Current log likelihood: [-6419.90669773]
Current log likelihood: [-6405.05127496]
Current log likelihood: [-6402.48308944]
Current log likelihoo

## Post estimation processing

We evaluate our parameter estimates using t-ratios (or p-Values).

In maximum likelihood estimation, we extract these from the variance-covariance matrix of the parameters.

The variance covariance matrix is not readily available but we need to calculate it.

This is done with the code below.

**DO NOT EDIT THE CHUNK OF CODE BELOW!**

In [41]:
# Vector of parameter estimates
parameters = result['x'] 

# Calculate hessian
print("Calculating Hessian, please wait (this may take a while...)")
Hess = nd.Hessian(SLL, method = 'forward')
hessian = Hess(parameters)
inv_hessian = np.linalg.inv(hessian)

# Parameter statistics
dof = Nobs - len(betas_start) - 1
se = np.sqrt(np.diag(inv_hessian)) # Standard errors
tratio = parameters/se # t-ratios
p_value = (1-t.cdf(np.abs(tratio),dof)) * 2 # p-values


# --- Sandwich estimator --- #

# The sandwich estimator provides the "robust" s.e., t-ratios and p-values.
# These should be preferred over the classical parameter statistics.

# We first need the gradients at the solution point
Grad = nd.Gradient(LL, method = 'forward')
gradient = Grad(parameters)

# Then we need to calculate the B matrix
B = np.array([])
for r in range(gradient.shape[0]):
    Bm = np.zeros([len(betas_start),len(betas_start)])
    gradient0 = gradient[r,:]
    for i in range(len(gradient0)):
            for j in range(len(gradient0)):
                element = gradient0[i]*gradient0[j]
                Bm[i][j] = element
    if B.size==0:
                    B = Bm
    else:
                    B = B+Bm

# Finally we "sandwich" the B matrix between the inverese hessian matrices
BHHH = (inv_hessian)@(B)@(inv_hessian)

print("... Done!!")

# Now it is time to calculate some "robust" parameter statistics
rob_se = np.sqrt(np.diag(BHHH)) # robust standard error
rob_tratio = parameters/rob_se # robust t-ratio
rob_p_value = (1-t.cdf(np.abs(rob_tratio),dof)) * 2 # robust p-value

Calculating Hessian, please wait (this may take a while...)
... Done!!


## Results
Finally, we got our results. Let's print them!

The outputs that we receive are:
* Estimates: These are the values of our parameters. We must check if the sign is consistent with our expectations
* s.e.: Standard errors of the parameters
* tratio: t-ratio of the parameters (significant if absolute value > 1.96)
* p_value: p-value of the parameters (significant if < 0.05)

The parameter statistics also have their **robust** versions. These should be preferred as they are less susceptible to weakly specified models.

In [43]:
arrays = np.column_stack((np.array(list(betas_start.keys())),parameters,se,tratio,p_value,rob_se,rob_tratio,rob_p_value))
results = pd.DataFrame(arrays, columns = ['Parameter','Estimate','s.e.','t-ratio0','p-value',
                                          'Rob s.e.','Rob t-ratio0','Rob p-value'])

results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']] = (
results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']].apply(pd.to_numeric, errors='coerce'))
numeric_cols = results.select_dtypes(include='number').columns
results[numeric_cols] = results[numeric_cols].round(3)
results # print results

,Parameter,Estimate,s.e.,t-ratio0,p-value,Rob s.e.,Rob t-ratio0,Rob p-value
0,asc,0.187,0.116,1.609,0.108,0.117,1.594,0.111
1,beta_dur,-0.004,0.002,-1.739,0.082,0.002,-1.739,0.082
2,beta_time,-0.238,0.022,-10.691,0.000,0.023,-10.225,0.000
3,beta_frag,0.950,0.047,20.108,0.000,0.047,20.115,0.000
4,beta_size,1.097,0.145,7.568,0.000,0.145,7.580,0.000
5,beta_compen,0.027,0.005,5.519,0.000,0.005,5.520,0.000


## Goodness-of-fit indices

Let's calculate some goodness-of-fit indices now (do not edit)

In [44]:
# First let's calculate the GoF indices

rho_squared = 1 - ((-result.fun)/(-SLL(np.zeros(len(betas_start)))))
adj_rho_squared = 1 - (((-result.fun)-len(betas_start))/(-SLL(np.zeros(len(betas_start)))))

AIC = 2*len(betas_start) - 2*(-result.fun)
BIC = len(betas_start)*np.log(Nobs) - 2*(-result.fun)

LL0t = "Log likelihood at zeros:" + str(-SLL(np.zeros(len(betas_start))))
LLinit = "Initial log likelihood:" + str(-SLL(np.array(list(betas_start.values()))))
LLfin = "Final log likelihood:" + str(-result.fun)

rs1 = "rho squared="+str(rho_squared)
rs2 = "adjusted rho squared="+str(adj_rho_squared)
ac = "AIC="+str(AIC)
bc = "BIC="+str(BIC)

# Then let's print the GoF indices

print(LL0t)
print(LLinit)
print(LLfin)

print(rs1)
print(rs2)
print(ac)
print(bc)

Log likelihood at zeros:[-6931.4718056]
Initial log likelihood:[-6931.4718056]
Final log likelihood:-6377.038541206292
rho squared=[0.07998781]
adjusted rho squared=[0.0791222]
AIC=12766.077082412585
BIC=12809.339124644443


## Save output

We can save our output using the code below (do not edit):

In [45]:
with open(f"{model_name}_results.txt", 'w') as f:
    f.write(f'{LL0t}\n')
    f.write(f'{LLinit}\n')
    f.write(f'{LLfin}\n')
    f.write(f'{rs1}\n')
    f.write(f'{rs2}\n')
    f.write(f'{ac}\n')
    f.write(f'{bc}\n')
    results.to_csv(f, index=False, sep='\t')
results.to_csv(f'{model_name}_results.csv', index=False)